In [ ]:
import pandas as pd
import numpy as np
import warnings
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Set up environment for reproducible results
random.seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')

print("Libraries imported and environment set up.")

Libraries imported and environment set up.


In [ ]:
# --- File paths ---
train_file_path = "/content/Train.csv"
test_file_path = '/content/Test.csv'

# Load the datasets
try:
    train_df = pd.read_csv(train_file_path)
    test_df = pd.read_csv(test_file_path)
except FileNotFoundError as e:
    print(f"Error loading files: {e}. Please ensure 'Train.csv' and 'Test.csv' are accessible.")
    raise

# Drop unnecessary index column ('Unnamed: 0')
if 'Unnamed: 0' in train_df.columns:
    train_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
if 'Unnamed: 0' in test_df.columns:
    test_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

# Target is the 'fall' column (Binary). The 'label' is dropped from features.
X_train = train_df.drop(['fall', 'label'], axis=1)
y_train = train_df['fall']
X_test = test_df.drop(['fall', 'label'], axis=1)
y_test = test_df['fall']

print(f"Training data shape: {X_train.shape} (Target: Binary Fall)")

Training data shape: (1428, 9) (Target: Binary Fall)


In [ ]:
# --- Action: KEEPING ALL 9 FEATURES ---

# Convert all feature columns to numeric, forcing errors to NaN.
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Handle NaNs and Infinities: Essential for the StandardScaler
for col in X_train.columns:
    X_train[col].replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test[col].replace([np.inf, -np.inf], np.nan, inplace=True)

    mean_val = X_train[col].mean()

    X_train[col].fillna(mean_val, inplace=True)
    X_test[col].fillna(mean_val, inplace=True)

print(f"Features used for training: {list(X_train.columns)} (9 features)")

Features used for training: ['acc_max', 'gyro_max', 'acc_kurtosis', 'gyro_kurtosis', 'lin_max', 'acc_skewness', 'gyro_skewness', 'post_gyro_max', 'post_lin_max'] (9 features)


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Overwrite X_train/X_test with scaled DataFrames for consistency
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("Features normalized.")

Features normalized.


In [ ]:
# Define the optimized grid of hyperparameters (Based on best binary results)
random_grid = {
    'n_estimators': [350, 450, 550],
    'max_features': ['sqrt'],
    'bootstrap': [True],
    'max_depth': [None, 60, 80],
    'min_samples_split': [6, 9, 12],
    'min_samples_leaf': [2, 3, 4],
}

# Initialize the base model
rf = RandomForestClassifier(random_state=42)

# Initialize and run RandomizedSearchCV
rf_random = RandomizedSearchCV(
    estimator = rf,
    param_distributions = random_grid,
    n_iter = 30,
    cv = 5,
    verbose=0,
    random_state=42,
    n_jobs = -1
)

print("Starting Optimized Binary Search...")
rf_random.fit(X_train, y_train)

best_params = rf_random.best_params_
print("\nSearch Complete.")
print("Best parameters found:")
print(best_params)

Starting Optimized Binary Search...

Search Complete.
Best parameters found:
{'n_estimators': 450, 'min_samples_split': 6, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 60, 'bootstrap': True}


In [ ]:
# Get the best estimator
best_estimator_binary = rf_random.best_estimator_

# Make predictions on the scaled test data
predictions = best_estimator_binary.predict(X_test)

# Calculate and print accuracy
optimal_accuracy = accuracy_score(y_test, predictions) * 100

print("-" * 50)
print("FINAL TIER 1 BINARY FALL DETECTION ACCURACY (Fall/Not Fall):")
print(f"Overall Accuracy = {optimal_accuracy:0.2f}%.")
print("-" * 50)


--------------------------------------------------
FINAL TIER 1 BINARY FALL DETECTION ACCURACY (Fall/Not Fall):
Overall Accuracy = 97.19%.
--------------------------------------------------


In [ ]:
import joblib
from google.colab import files

# --- 1. Define Filenames ---
MODEL_FILENAME_BINARY = "binary_fall_model.joblib"
SCALER_FILENAME_BINARY = "binary_scaler.joblib"

# --- 2. Get Your Trained Objects ---
# 'best_estimator_binary' is your best Random Forest model
# 'scaler' is your fitted StandardScaler object
# (Make sure these variables are still in memory from previous cells)
try:
    best_model = best_estimator_binary
    fitted_scaler = scaler
except NameError:
    print("❌ ERROR: 'best_estimator_binary' or 'scaler' not found.")
    print("Please re-run the training and scaling cells before this one.")
    raise

# --- 3. Save the Objects to Files ---
joblib.dump(best_model, MODEL_FILENAME_BINARY)
joblib.dump(fitted_scaler, SCALER_FILENAME_BINARY)

print(f"Binary model saved to: {MODEL_FILENAME_BINARY}")
print(f"Scaler saved to: {SCALER_FILENAME_BINARY}")

# --- 4. Download the Files ---
print("\nDownloading files to your local computer...")
files.download(MODEL_FILENAME_BINARY)
files.download(SCALER_FILENAME_BINARY)

Binary model saved to: binary_fall_model.joblib
Scaler saved to: binary_scaler.joblib



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import joblib
import numpy as np
from google.colab import files

# --- 1. UPLOAD FILES ---
print("Please upload your 'binary_fall_model.joblib' and 'binary_scaler.joblib' files.")
uploaded = files.upload()

# --- 2. CONFIGURATION ---
MODEL_FILENAME_BINARY = "binary_fall_model.joblib"
SCALER_FILENAME_BINARY = "binary_scaler.joblib"

# The 9 features your model was trained on
feature_names = [
    'acc_max',
    'gyro_max',
    'acc_kurtosis',
    'gyro_kurtosis',
    'lin_max',
    'acc_skewness',
    'gyro_skewness',
    'post_gyro_max',
    'post_lin_max'
]
EXPECTED_FEATURES = len(feature_names)

# --- 3. LOAD MODEL AND SCALER ---
try:
    model = joblib.load(MODEL_FILENAME_BINARY)
    scaler = joblib.load(SCALER_FILENAME_BINARY)
    print("\n✅ Binary Model and Scaler loaded successfully.")

except FileNotFoundError:
    print("\n❌ ERROR: Model/Scaler files not found. Please run Step 1 (upload) again.")
    raise

# --- 4. GET SENSOR READINGS FROM USER ---
print("\n" + "="*50)
print(f"Please enter the {EXPECTED_FEATURES} sensor readings one by one:")

features_list = [] # Store the 9 values here

for name in feature_names:
    while True: # Loop until a valid number is entered
        val_str = input(f"  Enter {name}: ")
        try:
            val_float = float(val_str)
            features_list.append(val_float)
            break # Good input, move to next feature
        except ValueError:
            print(f"  ❌ Invalid input. Please enter a number (e.g., 26.0)")

print("="*50)
print(f"\nInput readings collected: {features_list}")

# --- 5. PRE-PROCESS AND PREDICT ---
try:
    # a. Convert to a 2D NumPy array (shape (1, 9))
    input_array = np.array(features_list).reshape(1, -1)

    # b. Apply the loaded scaler
    scaled_input = scaler.transform(input_array)

    # c. Make the prediction (will be 0 or 1)
    prediction_value = model.predict(scaled_input)[0]

    # d. Get probabilities [prob_of_0, prob_of_1]
    probabilities = model.predict_proba(scaled_input)[0]
    confidence = np.max(probabilities) * 100

    # e. Show the result
    print("\n--- PREDICTION RESULT ---")
    if prediction_value == 1:
        print(f"Predicted Activity: FALL DETECTED (Class 1)")
    else:
        print(f"Predicted Activity: Not Fall (Class 0)")

    print(f"Confidence:         {confidence:.2f}%")

except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")

Please upload your 'binary_fall_model.joblib' and 'binary_scaler.joblib' files.


Saving binary_fall_model.joblib to binary_fall_model (1).joblib
Saving binary_scaler.joblib to binary_scaler (1).joblib

✅ Binary Model and Scaler loaded successfully.

Please enter the 9 sensor readings one by one:
  Enter acc_max: 10.19585514
  Enter gyro_max: 0.14197737
  Enter acc_kurtosis: 5.65221405
  Enter gyro_kurtosis: 5.182487071
  Enter lin_max: 0.081429176
  Enter acc_skewness: -0.01721634
  Enter gyro_skewness: 1.713976622
  Enter post_gyro_max: 0.086412199
  Enter post_lin_max: 0.081429176

Input readings collected: [10.19585514, 0.14197737, 5.65221405, 5.182487071, 0.081429176, -0.01721634, 1.713976622, 0.086412199, 0.081429176]

--- PREDICTION RESULT ---
Predicted Activity: Not Fall (Class 0)
Confidence:         99.97%
